In [ ]:
! pip install pandas --break-system-packages && pip install sqlalchemy --break-system-packages && pip install psycopg2-binary --break-system-packages

In [2]:
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv('../database/.env')

usuario_database = os.getenv('DB_USER')
senha_database = os.getenv('DB_PASSWORD')
host_database = os.getenv('DB_HOST')
port_database = os.getenv('DB_PORT')
name_database = os.getenv('DB_NAME')

DATABASE_URL = f'postgresql://{usuario_database}:{senha_database}@{host_database}:{port_database}/{name_database}'
engine = create_engine(DATABASE_URL,
                       connect_args={
                           "options": "-csearch_path=raw" # as tabelas serão aplicadas no schema chamado raw
                       })


Esse dicionário será usado para obter os arquivos brutos dos dados para posterior criação das tabelas *raw*

In [4]:
raw_data_dict = {
    '2021_receitas_raw_data' : 'data/recife-dados-receitas-2021.csv',
    '2022_receitas_raw_data' : 'data/recife-dados-receitas-2022.csv',
    '2023_receitas_raw_data' : 'data/recife-dados-receitas-2023.csv'
}

## Teste de conexão com o Banco de Dados

In [ ]:
try:    
    display(pd.read_sql('SELECT 1', engine))
    # Caso o output seja 0 [?column?] 1, conexão foi bem sucedida
except:
    print('A conexão com o banco falhou.')

,?column?
0,1


In [ ]:
for nome_tabela, caminho in raw_data_dict.items():
    try:
        # (Extract): Leitura da fonte e criação de um DataFrame pandas com as infos obtidas
        df_raw = pd.read_csv(caminho, on_bad_lines='warn', sep=';')

        # (Load): Carregamento direto, sem tratamento, do DataFrame no Banco de dados
        df_raw.to_sql(
            nome_tabela,
            con=engine,
            if_exists='replace',
            index=False
        )

    except Exception as e:
        print(f'erro no processamento do arquivo {caminho} : {e}')